# Decripition
- this is the simplified version only with catboost classifer and lightgbm regressor
- https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/discussion/566550

In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=9f8076ba5680562da4d8214dd62e129e0f2f91c433fd012de4dc85bfeddc24c3
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold,KFold,ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor,CatBoostClassifier
from lifelines.utils import concordance_index
from matplotlib import pyplot as plt
from lifelines import KaplanMeierFitter
from sklearn.metrics import roc_auc_score,mean_absolute_error
import os
from lifelines.utils import concordance_index
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score,mean_absolute_error
import itertools
from lightgbm import LGBMRegressor,LGBMClassifier
import warnings
import lightgbm as lgb
import optuna
from numba import jit
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
@jit(nopython=True)
def init_BTree(values):
    times_to_compare = np.empty_like(values)
    last_full_row = int(np.log2(len(values) + 1) - 1)
    len_ragged_row = len(values) - (2 ** (last_full_row + 1) - 1)
    if len_ragged_row > 0:
#        bottom_row_ix = np.s_[: 2 * len_ragged_row : 2]
        bottom_row_ix = slice(None, 2 * len_ragged_row, 2)
        times_to_compare[-len_ragged_row:] = values[bottom_row_ix]
        values = np.delete(values, bottom_row_ix)
    values_start = 0
    values_space = 2
    values_len = 2 ** last_full_row
    while values_start < len(values):
        times_to_compare[values_len - 1 : 2 * values_len - 1] = values[values_start::values_space]
        values_start += int(values_space / 2)
        values_space *= 2
        values_len = int(values_len / 2)
    return times_to_compare

@jit(nopython=True)
def insert(counts, pred, times_to_compare):
    i = 0
    n = len(times_to_compare)
    while (i < n):
        cur = times_to_compare[i]
        counts[i] += 1
        if pred < cur:
            i = 2 * i + 1
        elif pred > cur:
            i = 2 * i + 2
        else:
            return counts
    #raise ValueError("Value %s not contained in tree." "Also, the counts are now messed up." % times_to_compare)

@jit(nopython=True)
def fn_rank(pred, times_to_compare, counts):
    i = 0
    n = len(times_to_compare)
    rank = 0
    count = 0
    while (i < n):
        cur = times_to_compare[i]
        if pred < cur:
            i = 2 * i + 1
            continue
        elif pred > cur:
            rank += counts[i]
            # subtract off the right tree if exists
            nexti = 2 * i + 2
            if nexti < n:
                rank -= counts[nexti]
                i = nexti
                continue
            else:
                return rank, count
        else:  # value == cur
            count = counts[i]
            lefti = 2 * i + 1
            if lefti < n:
                nleft = counts[lefti]
                count -= nleft
                rank += nleft
                righti = lefti + 1
                if righti < n:
                    count -= counts[righti]
            return rank, count
    return rank, count


@jit(nopython=True)
def handle_pairs(truth, pred, first_ix, times_to_compare, counts):
    next_ix = first_ix
    while next_ix < len(truth) and truth[next_ix] == truth[first_ix]:
        next_ix += 1
    pairs = counts[0] * (next_ix - first_ix)
    correct = np.int64(0)
    tied = np.int64(0)
    for i in range(first_ix, next_ix):
#        rank, count = times_to_compare.rank(censored_pred[i])
        rank, count = fn_rank(pred[i], times_to_compare, counts)
        correct += rank
        tied += count
    return (pairs, correct, tied, next_ix)


@jit(nopython=True)
def fast_concordance_index(event_times, predicted_event_times, event_observed):
    
    died_mask = event_observed==1#.astype(bool)
    # TODO: is event_times already sorted? That would be nice...
    died_truth = event_times[died_mask]
    ix = np.argsort(died_truth)
    died_truth = died_truth[ix]
    died_pred = predicted_event_times[died_mask][ix]

    censored_truth = event_times[~died_mask]
    ix = np.argsort(censored_truth)
    censored_truth = censored_truth[ix]
    censored_pred = predicted_event_times[~died_mask][ix]

    censored_ix = 0
    died_ix = 0
    
    times_to_compare = init_BTree(np.unique(died_pred))
#    counts = np.zeros_like(times_to_compare, dtype=int)
    counts = np.full(len(times_to_compare), 0)
    
    num_pairs = np.int64(0)
    num_correct = np.int64(0)
    num_tied = np.int64(0)

    # we iterate through cases sorted by exit time:
    # - First, all cases that died at time t0. We add these to the sortedlist of died times.
    # - Then, all cases that were censored at time t0. We DON'T add these since they are NOT
    #   comparable to subsequent elements.
    while True:
        has_more_censored = censored_ix < len(censored_truth)
        has_more_died = died_ix < len(died_truth)
        # Should we look at some censored indices next, or died indices?
        if has_more_censored and (not has_more_died or died_truth[died_ix] > censored_truth[censored_ix]):
            pairs, correct, tied, next_ix = handle_pairs(censored_truth, censored_pred, censored_ix, times_to_compare, counts)
            censored_ix = next_ix
            
        elif has_more_died and (not has_more_censored or died_truth[died_ix] <= censored_truth[censored_ix]):
            pairs, correct, tied, next_ix = handle_pairs(died_truth, died_pred, died_ix, times_to_compare, counts)

            for pred in died_pred[died_ix:next_ix]:
                insert(counts, pred, times_to_compare)
                                
            died_ix = next_ix
        else:
            assert not (has_more_died or has_more_censored)
            break

        num_pairs += pairs
        num_correct += correct
        num_tied += tied
        
#    print(num_pairs, num_correct, num_tied)

    return (num_correct + num_tied / 2) / num_pairs

In [4]:
local=False

class global_args():
    def __init__(self,local):
        if local:
            self.data_dir='./data/'
            self.model_dir='./models/'
            self.data_process_dir='./data_process/'
            self.submission_dir='./submission/'
            self.encoder_info_dir='./encoder_info/'
            self.fold_n=5
        else:
            self.data_dir='/kaggle/input/equity-post-HCT-survival-predictions/'
            self.model_dir='/kaggle/working/models/'
            self.data_process_dir='/kaggle/working/data_process/'
            self.submission_dir='/kaggle/working/'
            self.encoder_info_dir='/kaggle/working/encoder_info/'
            self.fold_n=5


args = global_args(local=local)

if local==False:
    os.mkdir(args.model_dir)
    os.mkdir(args.data_process_dir)
    os.mkdir(args.encoder_info_dir)


def CIBMTR_score(y, y_hat,efs,race_group):
    merged_df = pd.DataFrame({'y':y,'y_hat':y_hat,'efs':efs,'race_group':race_group})
    merged_df = merged_df.reset_index(drop=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    race_list = []
    for race in merged_df_race_dict.keys():
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        c_index_race = fast_concordance_index(
                        np.array(merged_df_race['y']),
                        np.array(merged_df_race['y_hat']),
                        np.array(merged_df_race['efs']))
        metric_list.append(c_index_race)
        race_list.append(race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list))),np.sqrt(np.var(metric_list)),{race:cindex for race,cindex in zip(race_list,metric_list)}

# DataProcess

In [5]:
def feature_engineering(data_file,train):
    data = pd.read_csv(data_file)
    data_dict = pd.read_csv(args.data_dir+'data_dictionary.csv')
    feature_cat = list(data_dict.loc[data_dict.type=='Categorical','variable'])
    feature_cat = [x for x in feature_cat if x  not in ('efs','efs_time')]
    feature_value = list(data_dict.loc[data_dict.type=='Numerical','variable'])    
    feature_value = [x for x in feature_value if x  not in ('efs','efs_time')]

    for col in feature_value:
        if col not in ['donor_age','age_at_hct']:
            data[col+'_trans2cat'] = data[col].copy().astype('str')
            feature_cat.append(col+'_trans2cat')
    

    features = data[feature_cat].fillna('-1')
    for col in features.columns:
        features[col]=features[col].astype('str')

    features_new = data[feature_value]#.fillna(-1)
    features = pd.concat([features,features_new],axis=1)
    

    with open(args.encoder_info_dir+'one_hot_encoder.pkl','rb') as f:
        one_hot_encoder = pickle.load(f)
    
    # one-hot
    tmp = one_hot_encoder['features']
    features_new = one_hot_encoder['model'].transform(data[tmp]) 
    features_new=pd.DataFrame(features_new)
    features_new.columns = ['onehot_'+str(x) for x in range(len(features_new.columns))]
    features = pd.concat([features,features_new],axis=1)
    feature_onehot = list(features_new.columns)
    

    if train==True:
        return {'X':features,
                'efs':data.efs,
                'efs_time':data.efs_time,
                'feature_cat':feature_cat,
                'feature_value':feature_value,
                'feature_onehot':feature_onehot,
                'ID':data.ID,
                'race_group':data.race_group}
    else:
        return {'X':features,
                'feature_cat':feature_cat,
                'feature_value':feature_value,
                'feature_onehot':feature_onehot,
                'ID':data.ID,
                'race_group':data.race_group}


data = pd.read_csv(args.data_dir+'train.csv')
data_dict = pd.read_csv(args.data_dir+'data_dictionary.csv')


# one-hot encoder
one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_features = list(data_dict.loc[data_dict.type=='Categorical','variable'])
one_hot_features = [x for x in one_hot_features if x not in ('efs','efs_time')]
one_hot_encoder.fit(data[one_hot_features])
one_hot_encoder_info  ={'features':one_hot_features,'model':one_hot_encoder}
with open(args.encoder_info_dir+'one_hot_encoder.pkl','wb') as f:
    pickle.dump(one_hot_encoder_info, f)

# features engineering
data_train = feature_engineering(args.data_dir+'train.csv',train=True)
with open(args.data_process_dir+'data_train.pkl','wb') as f:
    pickle.dump(data_train,f)

data_test = feature_engineering(args.data_dir+'test.csv',train=False)
with open(args.data_process_dir+'data_test.pkl','wb') as f:
    pickle.dump(data_test,f)


##################features describe######################)
print('Categary Feature number:',len(data_train['feature_cat']))
print('numerical Feature number:',len(data_train['feature_value']))
print('One-hot Feature number:',len(data_train['feature_onehot']))

Categary Feature number: 55
numerical Feature number: 22
One-hot Feature number: 191


# Catboost Classifier Training

In [6]:
def cat_cls_seconde_process(data_file):
    """
    Data process for catboost classifier
    """
    with open(data_file,'rb') as f:
        data = pickle.load(f)
    for col in data['feature_cat']:
        data['X'][col] = data['X'][col].astype('category')
    input_columns = data['feature_cat']+data['feature_value']+data['feature_onehot']
    data['X'] = data['X'][input_columns]
    categories = {}
    for col in data['X'].columns:
        if data['X'][col].dtype=='category':
            categories[col] = data['X'][col].cat.categories
    return data,categories

def cat_cls_train(seed,fold_n):
    """
    Catboost Classifier training
    """
    data,categories = cat_cls_seconde_process(args.data_process_dir+'data_train.pkl')
    with open(args.encoder_info_dir+'./categories_cat_cls.pkl','wb') as f:
        pickle.dump(categories,f)
    data['cls_target'] = (data['efs']==0).astype('int')

    params = {'loss_function':'Logloss',
            'eval_metric':'AUC',
            'cat_features': data['feature_cat'],
            #'early_stopping_rounds': 100,
            'verbose': 1000,
            'random_seed': 43,
            'depth':6, 
            'learning_rate':0.02,
            'n_estimators':2000,
            'l2_leaf_reg':1,
            #'thread_count':11
            'subsample':0.66,
            'colsample_bylevel':0.8,
            }
    catbst_cls = CatBoostClassifier(**params)

    sample_num = len(data['X'])
    cls_prediction = np.zeros(sample_num)

    skf = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=seed)
    y_combine = data['efs'].astype('str')+'|'+data['X']['race_group'].astype('str')
    for i,(train_index,eval_index) in enumerate(skf.split(data['X'],y_combine)):
        print('############fold',i)

        X_train_cls = data['X'].iloc[train_index,:]
        y_train_cls = data['cls_target'][train_index]
        X_eval_cls = data['X'].iloc[eval_index,:]
        y_eval_cls = data['cls_target'][eval_index]


        eval_set =[(X_train_cls, y_train_cls),\
                (X_eval_cls, y_eval_cls)]  
        catbst_cls.fit(eval_set[0][0], eval_set[0][1], 
                eval_set=eval_set[1:2], 
                use_best_model=False
                )
        y_hat_cls = catbst_cls.predict_proba(X_eval_cls)[:,1]
        cls_prediction[eval_index]=y_hat_cls

        model_info = {'train_index':train_index,'eval_index':eval_index,'model':catbst_cls}
        with open(args.model_dir+'catboost_cls_fold%s.pkl'%(i),'wb') as f:
            pickle.dump(model_info,f)

    print('Over All OOF CLS-AUC:',roc_auc_score(data['cls_target'],cls_prediction))


cat_cls_train(888,args.fold_n)

############fold 0
0:	test: 0.6702781	best: 0.6702781 (0)	total: 212ms	remaining: 7m 2s
1000:	test: 0.7610635	best: 0.7610635 (1000)	total: 2m 8s	remaining: 2m 7s
1999:	test: 0.7621559	best: 0.7622603 (1666)	total: 4m 17s	remaining: 0us

bestTest = 0.7622603455
bestIteration = 1666

############fold 1
0:	test: 0.6873330	best: 0.6873330 (0)	total: 155ms	remaining: 5m 10s
1000:	test: 0.7580478	best: 0.7580677 (999)	total: 2m 7s	remaining: 2m 7s
1999:	test: 0.7597379	best: 0.7598889 (1969)	total: 4m 19s	remaining: 0us

bestTest = 0.7598888507
bestIteration = 1969

############fold 2
0:	test: 0.6697805	best: 0.6697805 (0)	total: 143ms	remaining: 4m 46s
1000:	test: 0.7606891	best: 0.7607442 (979)	total: 2m 13s	remaining: 2m 13s
1999:	test: 0.7611581	best: 0.7616295 (1582)	total: 4m 27s	remaining: 0us

bestTest = 0.761629532
bestIteration = 1582

############fold 3
0:	test: 0.6690374	best: 0.6690374 (0)	total: 148ms	remaining: 4m 56s
1000:	test: 0.7596601	best: 0.7596601 (1000)	total: 2m 9s	

# Lightgbm Regressor Training

In [7]:
def lgb_reg_seconde_process(data_file):
    """
    Data process for lightgbm classifier
    """

    with open(data_file,'rb') as f:
        data = pickle.load(f)
    for col in data['feature_cat']+data['feature_onehot']:
        data['X'][col] = data['X'][col].astype('category')
    input_columns = data['feature_cat']+data['feature_value']+data['feature_onehot']
    data['X'] = data['X'][input_columns]
    data['X'] = data['X'].drop(['year_hct_trans2cat','hla_match_a_high_trans2cat','hla_match_b_low_trans2cat','comorbidity_score_trans2cat'],axis=1)
    categories = {}
    for col in data['X'].columns:
        if data['X'][col].dtype=='category':
            categories[col] = data['X'][col].cat.categories
    return data,categories

def lgb_reg_train(seed,fold_n):
    # data process
    data,categories = lgb_reg_seconde_process(args.data_process_dir+'data_train.pkl')
    with open(args.encoder_info_dir+'./categories_lgb_reg.pkl','wb') as f:
        pickle.dump(categories,f)

    model_param = {
            'objective': 'regression',
            'min_child_samples': 20,
            'num_iterations': 10000,
            #'num_iterations': 40000,
            'learning_rate': 0.02,
            'extra_trees': True,
            'reg_lambda': 0,
            'reg_alpha': 0,
            'num_leaves': 128,
            'metric': 'mae',
            'max_depth': 6,
            'device': 'cpu',
            'max_bin': 128,
            'verbose': -1,
            'seed': 42,
            'num_threads':11
        }
    lgb_reg = LGBMRegressor(**model_param)

    # Create target: Grouped and normalized rank by efs as 
    efs_time_norm = data['efs_time'].copy()
    efs_time_norm[data['efs']==1] = data['efs_time'][data['efs']==1].rank()/sum(data['efs']==1)
    efs_time_norm[data['efs']==0] = data['efs_time'][data['efs']==0].rank()/sum(data['efs']==0)
    data['efs_time_norm'] = efs_time_norm

    # create sample weight
    sample_weight = np.zeros(len(data['efs']))
    sample_weight[data['efs']==1] = 0.6
    sample_weight[data['efs']==0] = 0.4
    data['sample_weight'] = sample_weight

    sample_num = len(data['X'])
    reg_prediction = np.zeros(sample_num)

    skf = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=seed)
    y_combine = data['efs'].astype('str')+'|'+data['X']['race_group'].astype('str')
    for i,(train_index,eval_index) in enumerate(skf.split(data['X'],y_combine)):
        print('############fold',i)
        # define data
        efs_train = data['efs'].iloc[train_index]
        efs_eval = data['efs'].iloc[eval_index]

        X_train_reg = data['X'].iloc[train_index,:]
        X_eval_reg =  data['X'].iloc[eval_index,:]
        # Add efs as extral feature
        X_train_reg['efs'] = data['efs'].iloc[train_index].astype('int')  
        X_eval_reg['efs'] = data['efs'].iloc[eval_index].astype('int')  
        X_train_reg['efs'] = X_train_reg['efs'].astype('category').cat.set_categories([0,1])
        X_eval_reg['efs'] = X_eval_reg['efs'].astype('category').cat.set_categories([0,1])
        
        y_train_reg = data['efs_time_norm'].iloc[train_index]
        y_eval_reg = data['efs_time_norm'].iloc[eval_index]
                
        eval_set = [(X_train_reg, y_train_reg),\
                    (X_eval_reg[efs_eval==1], y_eval_reg[efs_eval==1])]    # focus metric on efs==1
        
        # train
        lgb_reg.fit(eval_set[0][0], eval_set[0][1], 
                eval_set=eval_set[1:2], 
                sample_weight = data['sample_weight'][train_index],
                #eval_metric=eval_cindex,
                callbacks=[
                    #lgb.early_stopping(stopping_rounds=100),
                    #lgb.log_evaluation(period=1000),
                    ]
                )
        
        # OOF predict
        y_hat_reg = lgb_reg.predict(X_eval_reg) 
        reg_prediction[eval_index]=y_hat_reg

        # save model
        model_info = {'train_index':train_index,'eval_index':eval_index,'model':lgb_reg}
        with open(args.model_dir+'lightgbm_reg_fold%s.pkl'%(i),'wb') as f:
            pickle.dump(model_info,f)

    # calculate raw c-index on efs==1
    efs_index_1 = data['efs']==1
    c_index_overall = concordance_index(data['efs_time'][efs_index_1],reg_prediction[efs_index_1],data['efs'][efs_index_1])
    print('Over All OOF C-index where efs==1:',c_index_overall)

lgb_reg_train(888,args.fold_n)

############fold 0
############fold 1
############fold 2
############fold 3
############fold 4
Over All OOF C-index where efs==1: 0.7639301534318628


# Merge Regressor and Classifier

In [8]:
with open(args.data_process_dir+'data_train.pkl','rb') as f:
        data = pickle.load(f)
sample_num_train = len(data['X'])
with open(args.data_process_dir+'data_test.pkl','rb') as f:
        data = pickle.load(f)
sample_num_test = len(data['X'])
#####################################分类模型#######################################

def cls_predict(fold_n,test):
    # data process
    if test:
        data_file = args.data_process_dir+'data_test.pkl'
    else:
        data_file = args.data_process_dir+'data_train.pkl'


    data,_ = cat_cls_seconde_process(data_file)
    with open(args.encoder_info_dir+'categories_cat_cls.pkl','rb') as f:
        categories = pickle.load(f)
    for col in data['X'].columns:
        if data['X'][col].dtype=='category':
            data['X'][col] = data['X'][col].cat.set_categories(categories[col]) 


    output = np.zeros(len(data['X']))
    for i in range(fold_n):
        with open(args.model_dir+f'catboost_cls_fold{i}.pkl','rb') as f:
            model_info = pickle.load(f) 
        model = model_info['model']

        if test:
            y_hat = model.predict_proba(data['X'])[:,1]
            output+=y_hat/fold_n
        else:
            eval_index = model_info['eval_index']
            y_hat = model.predict_proba(data['X'].loc[eval_index])[:,1]            
            output[eval_index] = y_hat
    return output


#####################################回归模型#######################################
def reg_predict(fold_n,test):
    '''
    input:
        model_nm: xgboost/catboost/lightgbm
        fold_n: CV fold number when the model was training
        test: 
            True:inferece in the test dataset
            False: inferece the oof result in the train dataset
    output:
        predicted result of test data or oof
    '''

    # data process
    if test:
        data_file = args.data_process_dir+'data_test.pkl'
    else:
        data_file = args.data_process_dir+'data_train.pkl'
    data,_ = lgb_reg_seconde_process(data_file)
    with open(args.encoder_info_dir+'categories_lgb_reg.pkl','rb') as f:
        categories = pickle.load(f)
    for col in data['X'].columns:
        if data['X'][col].dtype=='category':
            data['X'][col] = data['X'][col].cat.set_categories(categories[col]) 

    data['X']['efs']=1
    data['X']['efs'] = data['X']['efs'].astype('category').cat.set_categories([0,1])

    output = np.zeros(len(data['X']))
    for i in range(fold_n):
        with open(args.model_dir+f'lightgbm_reg_fold{i}.pkl','rb') as f:
            model_info = pickle.load(f)
        model = model_info['model']

        if test:
            y_hat = model.predict(data['X'])
            output+=y_hat/fold_n
        else:
            eval_index = model_info['eval_index']
            output[eval_index] = model.predict(data['X'].loc[eval_index])
    return output

###################################### Search best merge param ######################################

def merge_fun(Y_HAT_REG,Y_HAT_CLS,a=2.96,b=1,c=0.52):
    y_fun = (Y_HAT_REG>0)*c*np.abs(Y_HAT_REG)**b
    x_fun = (Y_HAT_CLS>0)*np.abs(Y_HAT_CLS)**a
    res = (1-y_fun)*x_fun+y_fun
    
    # **make sure there is enough samples**
    res = pd.Series(res).rank()/len(res)
    return res


def combine_objective(trial,y_hat_reg,y_hat_cls,efs_time,efs,race_group):
    '''
    Objective funtion of optuna for search the best merge function params
    input:
        trial: optuna trial param
        y_hat_reg: regressor prediction
        y_hat_cls: classifier predction
        efs_time: efs_time of train dataset
        efs: efs of train dataset
        race_group: race group of train dataset
        cls_model_nm: classifier model name(set different search range for differen model)
    output:
        score: Stratified c-index
    '''
    params = {'Y_HAT_REG':y_hat_reg,
            'Y_HAT_CLS':y_hat_cls,
            'a':trial.suggest_uniform("a", 2, 3.5),
            'b':trial.suggest_uniform('b',0.5,1.5),  # could be setted as 1 for simplify
            'c':trial.suggest_uniform('c',0,1),
            }

    score,var_error,metric_list = CIBMTR_score(efs_time,merge_fun(**params),efs,race_group)
    return score


def merge_param_fit(efs,efs_time,race_group,y_hat_reg,y_hat_cls,fold_n,seed):
    '''
    Find the best param for model merge functiont with optuna and K-fold cross validation, given predicted cls result and reg result
    
    input:
        y_hat_reg: regressor prediction
        y_hat_cls: classifier predction
        efs_time: efs_time of train dataset
        efs: efs of train dataset
        race_group: race group of train dataset
        cls_model_nm: classifier model name(set different search range for differen model)      
    return:
        result_combine: oof merged result
        best_params: best param for merge function
    '''

    y_combine = pd.Series(efs).astype('str')+'|'+pd.Series(race_group).astype('str')
    skf = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=seed)
    result_combine = np.zeros(len(efs))
    
    best_params = []
    for i,(train_index,eval_index) in enumerate(skf.split(efs,y_combine)):
        study=optuna.create_study(direction='maximize')
        study.optimize(lambda trial: combine_objective(trial, 
                                                       y_hat_reg[train_index],
                                                       y_hat_cls[train_index],
                                                       efs_time[train_index],
                                                       efs[train_index],
                                                       race_group[train_index],
                                                       ), 
                                                       n_trials=200)
        
        result_combine[eval_index]=merge_fun(y_hat_reg[eval_index],y_hat_cls[eval_index],**study.best_params)
        best_params.append(study.best_params)
        print(f'Best merge param fold{i}:',study.best_params)
    return result_combine,best_params


def search_best_merge_params():
    with open(args.data_process_dir+'data_train.pkl','rb') as f:
        data = pickle.load(f)
        efs = np.array(data['efs'])
        efs_time = np.array(data['efs_time'])
        race_group = np.array(data['race_group'])


    y_hat_cls=cls_predict(fold_n=args.fold_n,test=False)
    auc = roc_auc_score(efs==0,y_hat_cls)
    print('cls auc:',auc)
    
    y_hat_reg=reg_predict(fold_n=args.fold_n,test=False)
    c_index = concordance_index(efs_time[efs==1],y_hat_reg[efs==1])
    print(f'reg c-index where efs==1:',c_index)
            

    print(f'search best merge param')
    # search the merge params and get the oof merge result 
    y_hat_merge,best_params = merge_param_fit(
        efs=efs,
        efs_time=efs_time,
        race_group=race_group,
        y_hat_reg=y_hat_reg,
        y_hat_cls=y_hat_cls,
        fold_n=5,
        seed=888
    )  

    c_index_overall,var_error,metric_list = CIBMTR_score(efs_time,y_hat_merge,efs,race_group)
    print(f'OOF Stratified C-index:',c_index_overall)

    return best_params


merge_param=search_best_merge_params()

# inference
y_hat_cls=cls_predict(fold_n=args.fold_n,test=True)
y_hat_reg=reg_predict(fold_n=args.fold_n,test=True)

y_hat_merge = []
for param in merge_param:
    y_hat_merge.append(merge_fun(y_hat_reg,y_hat_cls,**param))
y_hat_merge = np.mean(np.array(y_hat_merge).T,axis=1)

cls auc: 0.7610221258516461
reg c-index where efs==1: 0.7639301534318628
search best merge param
Best merge param fold0: {'a': 2.8176971060205265, 'b': 0.9956959106905089, 'c': 0.3640555139490638}
Best merge param fold1: {'a': 2.6882900421264573, 'b': 0.9398176725579255, 'c': 0.39208863838557917}
Best merge param fold2: {'a': 2.8235876806812215, 'b': 0.9584395665349859, 'c': 0.38156021131269563}
Best merge param fold3: {'a': 2.7175481124205727, 'b': 1.2154252676785475, 'c': 0.3979879581782422}
Best merge param fold4: {'a': 3.3016546707854824, 'b': 1.0555694235271247, 'c': 0.2871540746957343}
OOF Stratified C-index: 0.6918450031862138


# submission

In [9]:
pred = pd.DataFrame({'ID':data_test['ID'],'prediction':1-y_hat_merge})
pred.to_csv(args.submission_dir+'submission.csv',index=False)
print(pred)

      ID  prediction
0  28800    0.333333
1  28801    0.666667
2  28802    0.000000
